In [1]:
!pip install --quiet chardet
print('installed chardet; use chardet.detect')

installed chardet; use chardet.detect


In [2]:
from chardet import detect
filename = '/kaggle/input/classify-news-into-category/News Categoires.csv'

with open(file=filename, mode='rb') as input_fp:
    print(detect(input_fp.read()))

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [3]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer=filename, encoding='ISO-8859-1')
df['News60'] = df['News'].str[:60]
df['token count'] = df['News'].apply(lambda x: len(x.split()))
df.head()

,News,Category,News60,token count
0,The IMF now expects China's economy to grow by...,Economy,The IMF now expects China's economy to grow by...,29
1,Manufacturing activity in the Eurozone has dec...,Economy,Manufacturing activity in the Eurozone has dec...,22
2,Continued disruptions in the global supply cha...,Economy,Continued disruptions in the global supply cha...,24
3,Concerns about food security remain high due t...,Economy,Concerns about food security remain high due t...,25
4,"While some central banks, like the US Federal ...",Economy,"While some central banks, like the US Federal ...",28


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   News         252 non-null    object
 1   Category     252 non-null    object
 2   News60       252 non-null    object
 3   token count  252 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 8.0+ KB


In [5]:
!pip install --quiet sentence_transformers
print('installed sentence transformers package')

installed sentence transformers package


In [6]:
from plotly.express import histogram
histogram(data_frame=df, x='token count', color='Category')

In [7]:
from plotly.express import scatter
scatter(data_frame=df, x='Category', y='token count')

In [8]:
from arrow import now
from sentence_transformers import SentenceTransformer
MODEL = 'all-MiniLM-L12-v2' 
model_start = now()
model = SentenceTransformer(model_name_or_path=MODEL,)
# all our documents are short so the default max sequence length should be adequate
print('model max sequence length: {}'.format(model.max_seq_length))
embeddings = model.encode(df['News'].values.tolist())
print('model time: {}'.format(now() - model_start))

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

model max sequence length: 128


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

model time: 0:00:16.874767


In [9]:
from plotly.express import scatter
from umap import UMAP

umap_model = UMAP(n_components=2, random_state=2023, verbose=True, n_jobs=1)
df[['u0', 'u1']] = umap_model.fit_transform(X=embeddings)

scatter(data_frame=df, x='u0', y='u1', color='Category', hover_name='News60', height=800, size='token count')

UMAP(n_jobs=1, random_state=2023, verbose=True)
Sat Dec 30 19:31:04 2023 Construct fuzzy simplicial set
Sat Dec 30 19:31:05 2023 Finding Nearest Neighbors
Sat Dec 30 19:31:08 2023 Finished Nearest Neighbor Search
Sat Dec 30 19:31:11 2023 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Sat Dec 30 19:31:13 2023 Finished embedding


None of the categories are completely isolated, and frankly some of these categories are judgment calls: e.g. is an article about using AI in food production a Food article or an Artificial Intelligence article? And of course all international relations stories could be politics stories.